# Theory behind this transformation
### See pp.186-187 of D'Alessandro

We are interested in 2-qubit unitaries. Suppose that we ignore the global phase. Then w.l.o.g. our target unitary is a special unitary in $\mathsf{SU}(4)$. The corresponding Lie algebra is $\mathsf{su}(4)$.

We have a constand drift Hailtonian $H_D = \sigma_{z}\otimes\sigma_{z}$, whereas we can arbitrarily control the local Hamiltonians $H_C = \sum_{i=x,y,z}(a_{i}(t)\sigma_{i}\otimes I + b_i(t)I\otimes\sigma_{i})$ and coefficients $a_{i}(t),b_{i}(t)$ can be arbitrarily strong.

Then we naturally obtain the Cartan decomposition 
\begin{equation}
    \mathsf{su}(4) = \mathfrak{k} \oplus \mathfrak{p},
\end{equation}
where the subalgebra 
\begin{equation}
    \mathfrak{k} = \mathrm{span}\{i\sigma_{j}\otimes I, iI\otimes\sigma_{j} \vert j = x,y,z\}, 
\end{equation}
and
\begin{equation}
    \mathfrak{p} = \mathrm{span}\{i\sigma_{j}\otimes \sigma{k} \vert j,k = x,y,z\}.
\end{equation}
Note that 
\begin{equation}
    [ \mathfrak{k},\mathfrak{k} ] \subseteq \mathfrak{k},\quad [ \mathfrak{k},\mathfrak{p} ] \subseteq \mathfrak{p},\quad [ \mathfrak{p},\mathfrak{p} ] \subseteq \mathfrak{k}.
\end{equation}
The Lie group corresponding to the Lie algebra $\mathfrak{k}$ is $\mathcal{K} = \exp(\mathfrak{k})$.

Now we can define a Cartan subalgebra (included in $\mathfrak{p}$) w.r.t. the decomposition
\begin{equation}
    \mathfrak{a} = \mathrm{span}\{i\sigma_j\otimes\sigma_j \vert j = x,y,z\}.
\end{equation}
Then a decomposition 
\begin{equation}
    U = K_{1}AK_{2},
\end{equation}
where $K_1,K_2\in\mathcal{K}$ and $A \in \exp(\mathfrak{a})$ always exists. 
By finding the exponent of $A$, one can find the minimal time it takes to generate the target unitary $U$. 

It is known that $\mathfrak{k} = T \mathsf{so}(4) T^{-1}$, where $T$ is defined in the code below.
Hence, we first transform our unitary to be 
\begin{equation}
    \tilde{U} = T^{-1}UT = (T^{-1}K_1 T) (T^{-1} A T) (T^{-1} K_2 T) = O_1 D O_2,
\end{equation}
where $O_1, O_2$ are real orthogonal matrices with determinant 1 and $D$ is a diagonal matrix. 

Hence, if we find $D$, we can apply the inverse tranformation to get $A = TDT^{-1}$.

In [1]:
import numpy as np
from scipy.linalg import eig, det, sqrtm, inv, logm

# Groundworks

## Define $T$, $T^{-1}$, and a function to output the adjoint $T^{-1}UT$ (forward) or $TUT^{-1}$ (backward)

In [2]:
T = np.sqrt(1/2)*np.array([[0,1j,1,0],[1j,0,0,-1],[1j,0,0,1],[0,-1j,1,0]])
Tinv = np.transpose(np.conjugate(T))


def Tconj(mat,d = 'f'):
    if d=='f':
        return Tinv@mat@T
    else:
        return T@mat@Tinv

## Convert a general (4-dimensional) unitary into a special unitary 
### It only changes the global phase of a state which we do not care. If we need to care, use factor output

In [3]:
def UtoSU(mat):
    factor = np.log(det(mat),dtype = complex)/4
    newmat = mat/np.power(det(mat),1/4,dtype = complex)
    return newmat, factor

## Finding $O_1$ and $D$ 

We have $\tilde{U} = O_1DO_2$. Then $\tilde{U}\tilde{U}^{\top} = O_1D^2O_1^{\top}$.
Equivalently,
\begin{equation}
    \tilde{U}\tilde{U}^{\top} O_1 = O_1 D^2.
\end{equation}

Then $O_1$ is a matrix that has real orthonormal eigenvectors of $\tilde{U}\tilde{U}^{\top}$ as its columns and $D^2$ is a diagonal matrix whose diagonal entries are eigenvalues of $\tilde{U}\tilde{U}^{\top}$.

## This part is problematic!!! We need a better algorithm for $U = O_1DO_2$.

We cannot really recover $D$ that makes $O_2$ real and orthogonal.
See
\begin{equation}
D = \begin{pmatrix}
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 & -1
\end{pmatrix}
\end{equation}
which gives $D^2 = I$. Then the sqrtm function will give $D = I$ wrongly.

This wrong evaluation makes $\mathrm{det}(O_2) = -1$, and we don't want that. In such cases, we simply multiply -1 to the first diagonal element of $D$ and the first row of $O_2$. This makes $\mathrm{det}(O_2) = 1$. 

In [4]:
# from U, compute UU^{T}
def UUT(mat):
    return mat@np.transpose(mat)

In [5]:
# finding O_1 and D^2
def O1D2(UUTmat):
    w, v = eig(UUTmat)
    Diagmat = np.diag(w)
    return v, Diagmat

In [6]:
# output a candidate for O_1, D, O_2
def ODO_decomp(U_target):
    Utilde, factor = UtoSU(Tconj(U_target))
    O1, D2 = O1D2(UUT(Utilde))
    D1 = sqrtm(D2)
    O2 = inv(O1@D1)@Utilde
    if det(O2) != 1:
        D1 = np.diag([-1,1,1,1])@D1
        O2 = np.diag([-1,1,1,1])@O2
    return O1, D1, O2, factor

def KAK_decomp(U_target):
    O1, D, O2 , factor = ODO_decomp(U_target)
    return Tconj(O1,d = 'r'), Tconj(D,d = 'r'), Tconj(O2,d = 'r'), factor

From $A\in\exp(\mathfrak{a})$, 
\begin{equation}
    A = e^{ia} = e^{i(c_x \sigma_x\otimes\sigma_x +c_y \sigma_y\otimes\sigma_y + c_z \sigma_z\otimes\sigma_z)}.
\end{equation}
The minimal time is $c_x+c_y+c_z$.

To find $c_i$, use the inner product:
\begin{equation}
    c_i = \mathrm{Tr}[a(\sigma_i\otimes\sigma_i)]/4.
\end{equation}

In [7]:
sigx2 = np.kron(np.array([[0,1],[1,0]]),np.array([[0,1],[1,0]]))
sigy2 = np.kron(np.array([[0,-1j],[1j,0]]),np.array([[0,1],[1,0]]))
sigz2 = np.kron(np.array([[1,0],[0,-1]]),np.array([[1,0],[0,-1]]))
def optimal_drift(A):
    a = logm(A)/1j
    c_x = np.trace(a@sigx2)/4
    c_y = np.trace(a@sigy2)/4
    c_z = np.trace(a@sigz2)/4
    return c_x,c_y,c_z,np.abs(c_x)+np.abs(c_y)+np.abs(c_z)

## Example 

In [8]:
CZ = np.diag([1,1,1,-1])
K1, A, K2, factor = KAK_decomp(CZ)
cx, cy, cz, tmin = optimal_drift(A)
print('mintime: ',tmin)
print('drift term:', cx, ' for XX', cy, ' for YY', cz,' for ZZ')
print('control unitary before drift: \n', K1)
print('control unitary after drift: \n', K2)

mintime:  0.7853981633974483
drift term: 0j  for XX 0j  for YY (-0.7853981633974483+0j)  for ZZ
control unitary before drift: 
 [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]
control unitary after drift: 
 [[0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]]


In [9]:
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
K1, A, K2, factor = KAK_decomp(CNOT)
cx, cy, cz, tmin = optimal_drift(A)
print('mintime: ',tmin)
print('drift term:', cx, ' for XX', cy, ' for YY', cz,' for ZZ')
print('control unitary before drift: \n', K1)
print('control unitary after drift: \n', K2)

logm result may be inaccurate, approximate err = 3.9614610402651194286e-07
mintime:  0.7853981568012258
drift term: (-1.7212462011251262e-09-3.161014015820931e-08j)  for XX (-1.998824305824273e-08+9.46574096882813e-10j)  for YY (0.7853981051336119-5.268356067988802e-08j)  for ZZ
control unitary before drift: 
 [[ 0.70710678+0.00000000e+00j  0.        -1.01906619e-16j
  -0.70710678+3.68712595e-17j  0.        +0.00000000e+00j]
 [ 0.        -1.01906619e-16j  0.70710678+0.00000000e+00j
   0.        +0.00000000e+00j -0.70710678+3.68712595e-17j]
 [ 0.70710678-3.68712595e-17j  0.        +0.00000000e+00j
   0.70710678+0.00000000e+00j  0.        +1.01906619e-16j]
 [ 0.        +0.00000000e+00j  0.70710678-3.68712595e-17j
   0.        +1.01906619e-16j  0.70710678+0.00000000e+00j]]
control unitary after drift: 
 [[ 0.35355339+0.35355339j -0.35355339+0.35355339j  0.35355339-0.35355339j
   0.35355339+0.35355339j]
 [ 0.35355339+0.35355339j -0.35355339+0.35355339j -0.35355339+0.35355339j
  -0.35355339

In [10]:
SWAP = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
K1, A, K2, factor = KAK_decomp(SWAP)
cx, cy, cz, tmin = optimal_drift(A)
print('mintime: ',tmin)
print('drift term:', cx, ' for XX', cy, ' for YY', cz,' for ZZ')
print('control unitary before drift: \n', K1)
print('control unitary after drift: \n', K2)

logm result may be inaccurate, approximate err = 6.9601149183041838846e-07
mintime:  1.5707967237721268
drift term: (0.7853983546374321-2.688821387764051e-15j)  for XX (8.225095131722537e-08-4.681340370904508e-09j)  for YY (-0.7853982867506308+2.107342894703785e-08j)  for ZZ
control unitary before drift: 
 [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]
control unitary after drift: 
 [[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]


In [11]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import *
H = SymbolicHamiltonian( X(1)*X(2) + Y(1)*Y(2)+Z(1)*Z(2)+Z(1))
K1, A, K2, factor = KAK_decomp(CNOT)
cx, cy, cz, tmin = optimal_drift(A)
print('mintime: ',tmin)
print('drift term:', cx, ' for XX', cy, ' for YY', cz,' for ZZ')
print('control unitary before drift: \n', K1)
print('control unitary after drift: \n', K2)

[Qibo 0.2.9|INFO|2024-07-08 21:10:01]: Using tensorflow backend on /device:CPU:0


logm result may be inaccurate, approximate err = 3.9614610402651194286e-07
mintime:  0.7853981568012258
drift term: (-1.7212462011251262e-09-3.161014015820931e-08j)  for XX (-1.998824305824273e-08+9.46574096882813e-10j)  for YY (0.7853981051336119-5.268356067988802e-08j)  for ZZ
control unitary before drift: 
 [[ 0.70710678+0.00000000e+00j  0.        -1.01906619e-16j
  -0.70710678+3.68712595e-17j  0.        +0.00000000e+00j]
 [ 0.        -1.01906619e-16j  0.70710678+0.00000000e+00j
   0.        +0.00000000e+00j -0.70710678+3.68712595e-17j]
 [ 0.70710678-3.68712595e-17j  0.        +0.00000000e+00j
   0.70710678+0.00000000e+00j  0.        +1.01906619e-16j]
 [ 0.        +0.00000000e+00j  0.70710678-3.68712595e-17j
   0.        +1.01906619e-16j  0.70710678+0.00000000e+00j]]
control unitary after drift: 
 [[ 0.35355339+0.35355339j -0.35355339+0.35355339j  0.35355339-0.35355339j
   0.35355339+0.35355339j]
 [ 0.35355339+0.35355339j -0.35355339+0.35355339j -0.35355339+0.35355339j
  -0.35355339